# Майнор по Анализу Данных, Группа ИАД-2
## 24/05/2017  Рекомендательные системы

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

# Коллаборативная фильтрация

Метод коллаборативной фильтрации работает по принципу похожести пользователей (**User-based collaborative filtering**) или похожести товаров (**Item-based collaborative filtering**). 

Введем обозначения:
* $R_{ui}$ - оценка, которую дал пользователь $u$ товару $i$
* $\hat{R}_{ui}$ - прогноз оценки
* $s_{uv}$ - похожесть между пользователями $u$ и $v$
* $s_{ij}$ - похожесть между товарами $i$ и $j$
* $N(u)$ - Множество пользователей, наиболее похожих на $u$
* $N(i)$ - Множество товаров, наиболее похожих на $i$

Прогноз **user-based collaborative filtering** вычисляется по следующей формуле
$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv} R_{vi}}{\sum_{v \in N(u)} s_{uv}} $$

Походесть между пользователями можно вычислить разными способами, например с помощью корреляции или косинусной меры
$$ s_{uv} = \frac{R_u^T R_v}{\lVert R_u \rVert_2 \lVert R_v \rVert_2}$$
Стоит отметить, что при подсчете похожести должны учитываться только те товары, для которых и у $u$ и у $v$ есть какая-то оценка

<center><img src='http://comsysto.files.wordpress.com/2013/02/userbased.png' width=700></center>

Похожим образом делается оценка с помощью **item-based collaborative filtering**:

$$\hat{R}_{u i} = \frac{\sum_{j \in N(i)} s_{ij} R_{u j}}{\sum_{j \in N(i)} s_{ij}}$$

<center><img src='http://dataconomy.com/wp-content/uploads/2015/03/Beginners-Guide-Recommender-Systems-Content-Based-Filtering.png' width=350></center>

# Рейтинги фильмов

* Загрузите данные о рейтингах и фильмах
* Разбейте данные на обучение и контроль в пропорции 80/20
* Реализуйте функцию расчета попарных схожестей между пользователями
* Реализуйте метод расчета рекомендаций на основе $K$ наиболее похожих пользователей. Постройте график зафисимости ошибки MAE от $K$ (1-10)
* Выполните нормализацию рейтингов с помощью вычитания средней оценки $\bar{R}_u$ каждого из пользователей и повторите предыдущий 2 шага. В этом случае предсказание выполняется следующим образом
$$ \hat{R}_{ui} = \bar{R}_u + \frac{\sum_{v \in N(u)} s_{uv} (R_{vi} - \bar{R}_v)}{\sum_{v \in N(u)} s_{uv}} $$
* Перейдите к Item-Based подходу и повторите предыдущие шаги

In [14]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [15]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

In [16]:
movie_counts = df_rates.movieID.value_counts()

In [17]:
movies_one_watch = movie_counts[movie_counts == 1]

In [18]:
idx = df_rates.movieID.isin(movies_one_watch.index)
df_rates = df_rates.loc[~idx, :]

## Перекодируем ID фильмов

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
mov_enc = LabelEncoder()
mov_enc.fit(df_rates.movieID.values)
n_movies = df_rates.movieID.nunique()

In [21]:
user_enc = LabelEncoder()
user_enc.fit(df_rates.userID.values)
n_users = df_rates.userID.nunique()

In [22]:
df_rates.loc[:, 'movieID'] = mov_enc.transform(df_rates.movieID.values)
df_rates.loc[:, 'userID'] = user_enc.transform(df_rates.userID.values)

## Разбиваем на обучение и контроль

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
df_rates_train, df_rates_test = train_test_split(df_rates, 
                                                 test_size=0.2, 
                                                 random_state=123)

In [25]:
df_rates_test.shape

(171000, 9)

## Делаем матрицу с рейтингами

In [27]:
from scipy.sparse import coo_matrix

In [67]:
n_users_train = df_rates.userID.nunique()
R_train = coo_matrix((df_rates_train.rating, 
                     (df_rates_train.userID.values, df_rates_train.movieID.values)),
                     shape=(n_users, n_movies))

## Считаем попарные рейтинги

In [86]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.spatial.distance import correlation
from sklearn.metrics import pairwise_distances